# ARTEMIS

- [ ] Why sometimes we don't have pressure tensor but have temperature tensor?

- THB_L2_MOM

  - `thb_peem_ptens_magQ` -----Electron Pressure Tensor, Field Aligned

  - `thb_peem_t3_magQ` -----Electron Temperature, Field Aligned (TprpFA1, TprpFA2, TparFA)

  - `thb_peim_ptens_magQ` -----Ion Pressure Tensor, Field Aligned

  - `thb_peim_t3_magQ` -----Ion Temperature, Field Aligned (TprpFA1, TprpFA2, TparFA)


In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from utils import IDsConfig, WindConfig, THEMISConfig
from rich import print

In [36]:
enc = 2
# timerange = ["2019-04-10", "2019-04-10T01"]
timerange = ["2019-04-09", "2019-04-12"]
dir = f"../data/enc{enc}/"

wind_conf = WindConfig(
    timerange =timerange,
    tau=30,
)

thm_conf = THEMISConfig(
    timerange =timerange,
    tau=30,
)

# fname = f"wind_ids_dataset_{enc}"
confs : list[IDsConfig] = [wind_conf, thm_conf]
print(wind_conf)
print(thm_conf)

WindConfig(
    tau=datetime.timedelta(seconds=30),
    ts=datetime.timedelta(microseconds=90909),
    timerange=['2019-04-09', '2019-04-12'],
    mag_meta=Meta(dataset='WI_H2_MFI', parameters=['BGSE']),
    plasma_meta=PlasmaMeta(
        dataset='WI_PM_3DP',
        parameters=['P_DENS', 'P_VELS', 'P_TEMP'],
        density_col=None,
        velocity_cols=None,
        speed_col=None,
        temperature_col=None
    ),
    ion_temp_meta=TempMeta(
        dataset='WI_PLSP_3DP',
        parameters=['MOM.P.MAGT3'],
        para_col='proton_MagT3_Z',
        perp_cols=['proton_MagT3_X', 'proton_MagT3_Y']
    ),
    e_temp_meta=TempMeta(
        dataset='WI_ELM2_3DP',
        parameters=['MAGT3'],
        para_col='electron_MagT3_Para',
        perp_cols=['electron_MagT3_Perp1', 'electron_MagT3_Perp2']
    ),
    fname='wind_ids_ds',
    fmt='arrow'
)

THEMISConfig(
    tau=datetime.timedelta(seconds=30),
    ts=datetime.timedelta(seconds=1),
    timerange=['2019-04-09', '2019-04-12'],
    mag_meta=Meta(dataset='THB_L2_FGM', parameters=['thb_fgl_gse']),
    plasma_meta=PlasmaMeta(
        dataset='THB_L2_MOM',
        parameters=['thb_peim_densityQ', 'thb_peim_velocity_gseQ', 'thb_peim_ptotQ'],
        density_col=None,
        velocity_cols=None,
        speed_col=None,
        temperature_col=None
    ),
    ion_temp_meta=TempMeta(
        dataset='THB_L2_MOM',
        parameters=['thb_peim_t3_magQ'],
        para_col='Tz_ion FA MOM ESA-B',
        perp_cols=['Tx_ion FA MOM ESA-B', 'Ty_ion FA MOM ESA-B']
    ),
    e_temp_meta=TempMeta(
        dataset='THB_L2_MOM',
        parameters=['thb_peem_t3_magQ'],
        para_col='Tz_elec FA MOM ESA-B',
        perp_cols=['Tx_elec FA MOM ESA-B', 'Ty_elec FA MOM ESA-B']
    ),
    fname='thm_ids_ds',
    fmt='arrow'
)

In [37]:
from utils import calc_pressure_anisotropy

for conf in confs:
    ids = conf.get_and_process_data().find_events().update_candidates_with_plasma_data()
    path = f"{dir}/{conf.fname}.{conf.fmt}"
    
    ids.events = ids.events.sort("time").join_asof(
        conf.e_temp_df.collect().sort("time"), on ="time"
    ).join_asof(
        conf.ion_temp_df.collect().sort("time"), on ="time"
    ).pipe(calc_pressure_anisotropy)
    
    ids.export(path)

22-Feb-24 09:18:59: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

Estimated completion of line 4:   0%           Elapsed time: 00:00, estimated remaining time: ?

22-Feb-24 09:19:02: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

In [ ]:
# | label: fig-data-products
import speasy as spz
from speasy import SpeasyVariable
import matplotlib.pyplot as plt

products = [
    "cda/THB_L2_MOM/thb_peem_t3_magQ",
    "cda/THB_L2_MOM/thb_peim_t3_magQ",
    "cda/WI_ELM2_3DP/MAGT3",
    # "cda/WI_PLSP_3DP/MOM.P.MAGF",
]

In [ ]:
earth_start = "2019-04-09"
earth_end = "2019-04-12"
timerange = (earth_start, earth_end)